In [1]:
import os
from pathlib import Path
import time 

midi_folder = "\data\midis"

song = os.getcwd() + midi_folder + r'\No-Scrubs.mid'

testing midi playback and extraction

In [2]:
from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced

midi_file = song

MIDIPlayer(song, 600, styler=cifka_advanced, title='My Player')


to encode the songs, we need to
- extract the relevant streams out of them; 2 or 16-bar melody/trio
    - 16 seems more whole-song
    - for melody, maybe https://github.com/xinyiguan/MIDI_Melody_Extraction/blob/master/MIDI_MelodyExtractionTutorial.ipynb
    - or maybe magenta pipelines
    - https://web.mit.edu/music21/
- run through the conversion to note sequences
- run through the relevant encoding model
    - need to download from magenta
    - then try and decode?

to see if this all works, can run no scrubs for example through the pipeline

- midis

- convert to tf record with convert directory/convert file (https://github.com/magenta/magenta/blob/main/magenta/scripts/convert_dir_to_note_sequences.py#L128)
or convert a single midi to a notesequence https://github.com/magenta/note-seq/blob/main/note_seq/midi_io.py#L173

**question**: do we need to quantise/extract melody?
hypothesis - no. 

- so try the encoding method from the trained model (https://github.com/magenta/magenta/blob/main/magenta/models/music_vae/trained_model.py)




trying class code

In [3]:
from src.embeddings.music_vae.encoder import MusicVAEncoder
import os

midi_folder = "\data\midis"
no_scrubs_path = os.getcwd() + midi_folder + r'\No-Scrubs.mid'

nse = MusicVAEncoder(no_scrubs_path)


c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find 

Loaded MIDI file: c:\Users\cunn2\OneDrive\DSML\Project\thesis-repo\data\midis\No-Scrubs.mid


In [4]:
nse.extract_melodies()

QuantizationStatusError: NoteSequence  is not quantized or is quantized based on absolute timing.

using the notebook

In [ ]:
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow as tf

import os
from pathlib import Path
from dotenv import load_dotenv
import time 

load_dotenv()
base = os.environ['ROOT_PATH']
midis_path = base + r'\music_vae\midis'
models = base + r'\music_vae\models'

mel_2bar_config = configs.CONFIG_MAP['cat-mel_2bar_big']
mel_2bar = TrainedModel(mel_2bar_config, batch_size=4, checkpoint_dir_or_path=models + '\cat-mel_2bar_big.ckpt')

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

INFO:tensorflow:Restoring parameters from C:\Users\cunn2\OneDrive\DSML\Project\music-project\music_vae\models\cat-mel_2bar_big.ckpt


In [ ]:
midi_files = ['No-Scrubs.mid']

print(midi_files)

# mel_input_seqs = [mm.midi_to_sequence_proto(m) for m in midi_files]

mel_input_seqs = []
for midi_file in midi_files:
    file_path = os.path.join(midis_path, midi_file)
    try:
        # Read the MIDI file as bytes
        with open(file_path, 'rb') as file:
            midi_data = file.read()
        # Convert the bytes data to a NoteSequence
        mel = mm.midi_to_sequence_proto(midi_data)
        mel.filename = midi_file
        mel_input_seqs.append(mel)
    except Exception as e:
        print(f"Failed to process {midi_file}: {str(e)}")

extracted_mels = []
for ns in mel_input_seqs:
  extracted_mels.extend(
      mel_2bar_config.data_converter.from_tensors(
          mel_2bar_config.data_converter.to_tensors(ns)[1]))
  

# for i, ns in enumerate(extracted_mels):
#   print("Melody", i)
#   play(ns)

['No-Scrubs.mid']


In [ ]:
extracted_mels[0
               ]

ticks_per_quarter: 220
tempos {
  qpm: 120.0
}
notes {
  pitch: 63
  velocity: 80
  start_time: 1.75
  end_time: 2.375
}
notes {
  pitch: 59
  velocity: 80
  start_time: 2.375
  end_time: 2.75
}
notes {
  pitch: 56
  velocity: 80
  start_time: 2.75
  end_time: 3.0
}
notes {
  pitch: 59
  velocity: 80
  start_time: 3.0
  end_time: 3.375
}
notes {
  pitch: 56
  velocity: 80
  start_time: 3.375
  end_time: 3.75
}
total_time: 3.75

In [ ]:
len(extracted_mels)

5

In [ ]:
mel = extracted_mels[1]

note_sequence_to_midi_file(mel, midi_file_loc)

from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced

song = midi_file_loc

MIDIPlayer(song, 400, styler=cifka_advanced, title='My Player')

In [ ]:
z = mel_2bar.encode([mel])

In [ ]:
recon = mel_2bar.decode(z[0], length=50)[0]

In [ ]:

note_sequence_to_midi_file(recon, midi_file_loc)

from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced

song = midi_file_loc

MIDIPlayer(song, 400, styler=cifka_advanced, title='My Player')

testing utils

In [ ]:
import utils as u
import os
from pathlib import Path
from dotenv import load_dotenv
import time 

load_dotenv()
base = os.environ['ROOT_PATH']
midis_path = base + r'\music_vae\midis'

song = midis_path + r'\No-Scrubs.mid'

segments = u.split_into_2_bar_segments(song)

In [ ]:
segments[0].total_time

5.14918325

In [ ]:
from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced
from note_seq.midi_io import midi_file_to_note_sequence, note_sequence_to_midi_file


testloc = base + r'\music_vae\test.mid'

note_sequence_to_midi_file(segments[0], testloc)

MIDIPlayer(testloc, 400, styler=cifka_advanced, title='My Player')

In [ ]:
a = [1,2,4,5,67]
print(a[::2])

[1, 4, 67]


In [ ]:
a = [1,3,5,7,9]
b = [2,4,6,8,10]

c = [0 for i in range (len(a)+len(b))]

c[::2], c[1::2] = a, b

print(c)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
c = []
c[4] = 3

IndexError: list assignment index out of range